In [ ]:
import pandas as pd
import plotly.graph_objects as go
import shared

pd.options.plotting.backend = "plotly"
pd.set_option('display.max_columns', None)

# it's important to define index and parse_dates
landkreise = pd.read_csv('data/inf_lk.csv', sep=',', index_col=0, parse_dates=True)
lander = pd.read_csv('data/inf_bl.csv', sep=',', index_col=0, parse_dates=True)

palette = shared.get_palette()

In [ ]:
ags_df = pd.read_json('data/ags.json').transpose()


def get_ags(lk_name):
    row = ags_df[ags_df['name'] == lk_name]
    return f'{row.index.values[0]}'


def get_lk_pop(lk_name):
    row = ags_df[ags_df['name'] == lk_name]
    return row.population.values[0]


# get land pop can be found in shared.py

def get_state(lk_name):
    row = ags_df[ags_df['name'] == lk_name]
    return row.state.values[0]


ags_df[ags_df['name'] == 'SK München']

In [ ]:
lander.head(3)

In [ ]:
def trend(x):
    if x < -0.05:
        return 'DOWN_STRONG'
    elif x < -0.01:
        return 'DOWN'
    elif x < 0.01:
        return 'STEADY'
    elif x < 0.05:
        return 'UP'
    else:
        return 'UP_STRONG'

def week_inz_lk(grouped_df, year_and_week):
    return round(grouped_df.loc[year_and_week, 'inz'],2)

def prepare_lk(lk_name):
    lk = landkreise[get_ags(lk_name)].to_frame()
    lk = lk.rename(columns={f"{get_ags(lk_name)}": "inf_sum"})

    lk['dif'] = lk.inf_sum - lk.inf_sum.shift(1)
    lk['dif'] = lk['dif'].fillna(0)
    lk = lk.astype({'dif': 'int64'})

    lk['rolling_dif'] = lk.dif.rolling(7).mean()

    lk['inf_7_tage'] = lk.inf_sum - lk.inf_sum.shift(7)
    lk['inf_7_tage'] = lk['inf_7_tage'].fillna(method='bfill')
    lk = lk.astype({'inf_7_tage': 'int64'})

    lk['inz'] = round(lk['inf_7_tage'] / (get_lk_pop(lk_name) / 100_000), 1)

    lk['inz_dif'] = (lk['inz'] - lk['inz'].shift(1))
    lk['inz_trend'] = round(lk['inz_dif'] / lk.inz, 2)
    lk['inz_trend'] = lk.apply(lambda x: trend(x['inz_trend']), axis=1)

    lk['date'] = lk.index
    lk = shared.add_weekday_stuff(lk, 'date')
    grouped = lk.groupby(['year_and_week']).mean()
    lk['week_inz'] = lk.apply(lambda x: week_inz_lk(grouped, x['year_and_week']), axis=1)

    return lk


In [ ]:
def week_inz_bl(kurzel, grouped_df, year_and_week):
    return round(grouped_df.loc[year_and_week, f'{kurzel}_inz'],2)

def prepare_lander(df):
    for kurzel in shared.bl_kurzel:
        df[f'{kurzel}_sum'] = df[shared.inf_lander_dict[kurzel]]
        df = df.drop(columns=[shared.inf_lander_dict[kurzel]])

    # add sum_diff
    for kurzel in shared.bl_kurzel:
        df[f'{kurzel}_dif'] = df[f'{kurzel}_sum'] - df[f'{kurzel}_sum'].shift(1)
        df[f'{kurzel}_dif_7'] = df[f'{kurzel}_sum'] - df[f'{kurzel}_sum'].shift(7)
    # add inzidenz
    for kurzel in shared.bl_kurzel:
        df[f'{kurzel}_inz'] = round(df[f'{kurzel}_dif_7'] * 100_000 / shared.get_land_pop(kurzel), 2)
    # more
    df['date'] = df.index
    df = shared.add_weekday_stuff(df, 'date')
    # trend
    for kurzel in shared.bl_kurzel:
        df[f'{kurzel}_inz_dif'] = df[f'{kurzel}_inz'] - df[f'{kurzel}_inz'].shift(1)
        df[f'{kurzel}_inz_trend'] = round(df[f'{kurzel}_inz_dif'] / df[f'{kurzel}_inz'], 2)
        df[f'{kurzel}_inz_trend'] = df.apply(lambda x: trend(x[f'{kurzel}_inz_trend']), axis=1)
        
    for kurzel in shared.bl_kurzel:
        grouped = df.groupby(['year_and_week']).mean()
        df[f'{kurzel}_week_inz'] = df.apply(lambda x: week_inz_bl(kurzel, grouped, x['year_and_week']), axis=1)

    return df


lander = prepare_lander(lander)

In [ ]:
grouped = lander.groupby(['year_and_week']).mean()
round(grouped.loc['2021_07', 'BY_inz'],2)

In [ ]:
lander.tail(3)

In [ ]:
def create_inz_fig():
    palette = shared.get_palette()
    fig = go.Figure()
    fig.update_layout(
        width=1050,
        height=600,
        title='Inzidenz'
    )
    return fig


def add_bl_inz(fig, lander_kurzel, smooth=1):
    for kurzel in lander_kurzel:
        name = shared.bl_dict[kurzel] if smooth == 1 else f'{shared.bl_dict[kurzel]} (Rolling {smooth} day window)'
        fig.add_trace(go.Scatter(x=lander.index,
                              y=lander[f'{kurzel}_inz'].rolling(smooth).mean(),
                              mode='lines',
                              name=name,
                              marker_color=next(palette),
                              line=dict(width=3)))
    return fig


def add_lk_inz(fig, lk_namen, smooth=1):
    for lk in lk_namen:
        lk_df = prepare_lk(lk)
        name = lk if smooth == 1 else f'{lk} (Rolling {smooth} day window)'
        fig.add_trace(go.Scatter(x=lk_df.index,
                              y=lk_df['inz'].rolling(smooth).mean(),
                              mode='lines',
                              name=name,
                              marker_color=next(palette),
                              line=dict(width=2)))
    return fig


def add_bl_cal_inz(fig, lander_kurzel):
    for kurzel in lander_kurzel:
        fig.add_trace(go.Scatter(x=lander.index,
                              y=lander[f'{kurzel}_week_inz'],
                              name=f'{shared.bl_dict[kurzel]} (Ø Woche)',
                              marker_color=next(palette),
                              line_shape='hv',
                              line=dict(dash='dot', width=2)))
    return fig


def add_lk_cal_inz(fig, lk_namen):
    for lk in lk_namen:
        lk_df = prepare_lk(lk)
        #lk_df = lk_df.groupby(['year_and_week']).mean()
        fig.add_trace(go.Scatter(x=lk_df.index,
                              y=lk_df['week_inz'],
                              name=f'{lk} (Ø Woche)',
                              marker_color=next(palette),
                              line_shape='hv',
                              line=dict(dash='dot', width=2)))
    return fig



In [ ]:
fig = create_inz_fig()
fig = add_bl_inz(fig, lander_kurzel=shared.bl_kurzel)
fig.update_layout(title='Inzidenz Bundesländer')
fig.show()

In [ ]:
fig = create_inz_fig()
fig = add_bl_inz(fig, lander_kurzel=['BY', 'DE'])
fig = add_lk_inz(fig, lk_namen=['SK München', 'LK München'])
fig.update_layout(title='Inzidenz München')
fig.show()

In [ ]:
fig = create_inz_fig()
fig = add_bl_inz(fig, lander_kurzel=['DE'])
fig = add_lk_inz(fig,
                 lk_namen=['SK München', 'Berlin', 'SK Flensburg', 'LK Tirschenreuth', 'LK Regensburg', 'LK Miesbach'])
fig.update_layout(title='Inzidenz Bundesländer')
fig.show()


In [ ]:
fig = create_inz_fig()
fig = add_bl_inz(fig, lander_kurzel=['DE'], smooth=21)
fig = add_bl_inz(fig, lander_kurzel=['DE'], smooth=7)
fig = add_bl_inz(fig, lander_kurzel=['DE'], smooth=1)
fig = add_lk_inz(fig, lk_namen=['SK München'], smooth=1)
fig = add_lk_inz(fig, lk_namen=['SK München'], smooth=7)
fig = add_lk_inz(fig, lk_namen=['SK München'], smooth=21)

fig.update_layout(title='Inzidenz')
fig.show()



In [ ]:
muc_90 = prepare_lk('SK München').tail(90)


In [ ]:
row = muc_90[muc_90['inz'] == min(muc_90['inz'])]
row

In [ ]:
muc = prepare_lk('SK München').tail(365)
muc[muc.inz == muc.inz.min()]

In [ ]:
muc = prepare_lk('SK München')
muc[muc.inz == muc.inz.max()]


In [ ]:
muc_recent = prepare_lk('SK München').tail(50)
muc_recent[muc_recent['inz'] == max(muc_recent['inz'])]

In [ ]:
fig = create_inz_fig()
palette = shared.get_palette()
fig = add_bl_inz(fig, ['DE','BY'])
palette = shared.get_palette()
fig = add_bl_cal_inz(fig, ['DE', 'BY'])

fig.show()

In [ ]:
fig = create_inz_fig()
palette = shared.get_palette()
fig = add_bl_inz(fig, ['DE','BY'])
fig = add_lk_inz(fig, ['SK München', 'LK München', 'LK Miesbach'])
palette = shared.get_palette()
fig = add_bl_cal_inz(fig, ['DE', 'BY'])
fig = add_lk_cal_inz(fig, ['SK München', 'LK München', 'LK Miesbach'])
shared.write_html(fig, f'inz_DE-BY-MUC')
fig.show()

In [ ]:

def plot_lk_inz_per_week(*lk_namen):
    fig = go.Figure()
    for lk in lk_namen:
        lk_df = prepare_lk(lk)
        lk_df = lk_df.groupby(['year_and_week']).mean()
        fig.add_trace(go.Bar(x=lk_df.index,
                             y=lk_df['inz'],
                             name=lk, ))

    fig.update_layout(
        width=1000,
        height=500,
        title=f'Durchschnitt der Inzidenz pro Kalenderwoche'
    )
    fig.show()


plot_lk_inz_per_week('SK München', 'LK München', 'LK Tirschenreuth')

### Inzidenz Geglättet

In [ ]:

def plot_lk_inz_smooth(*lk_namen, smooth=7):
    fig = go.Figure()
    for lk in lk_namen:
        lk_df = prepare_lk(lk)

        fig.add_trace(go.Scatter(x=lk_df.index,
                                 y=lk_df['inz'].rolling(smooth).mean(),
                                 mode='lines',
                                 name=lk,
                                 visible="legendonly",
                                 line=dict(width=2)))

    fig.update_layout(
        width=1000,
        height=500,
        title=f'Inzidenz geglättet ({smooth} Tage Fenster)'
    )
    fig.show()


plot_lk_inz_smooth('SK München', 'LK München', 'Berlin', 'SK Flensburg', 'LK Tirschenreuth', 'LK Regensburg',
                   'LK Miesbach', smooth=21)


### Änderung der Inzidenz

In [ ]:
def plot_lk_inz_change(*lk_namen, smooth=7):
    fig = go.Figure()
    for lk in lk_namen:
        lk_df = prepare_lk(lk)

        fig.add_trace(go.Scatter(x=lk_df.index,
                                 y=lk_df['inz_dif'].rolling(smooth).mean(),
                                 mode='lines',
                                 name=lk,
                                 line=dict(width=2)))

        fig.add_trace(go.Scatter(x=lk_df.index,
                                 y=lk_df['inz_dif'],
                                 mode='markers',
                                 name=lk,
                                 marker=dict(
                                     color='grey',
                                     size=2,
                                 )))

    fig.update_layout(
        width=1000,
        height=500,
        title=f'Änderung der Inzidenz Geglättet ({smooth} Tage Fenster)'
    )
    fig.show()


plot_lk_inz_change('SK München', 'Berlin', 'SK Flensburg', 'LK Tirschenreuth', 'LK Regensburg', 'LK Miesbach',
                   smooth=21)
